In [ ]:
!git clone https://github.com/dreamgaussian/dreamgaussian

In [ ]:
%pip install xformers==0.0.19 torch==2.0.0+cu118 torchvision  torchaudio --extra-index-url https://download.pytorch.org/whl/cu118

In [ ]:
!sudo apt-get update
!sudo apt-get install libeigen3-dev libegl1-mesa-dev xvfb -y 

In [ ]:
!git clone https://github.com/huggingface/transformers.git
%cd transformers
!git pull
%pip install -e .

In [ ]:
%cd /notebooks/dreamgaussian
!git pull

In [ ]:
%pip install -r requirements.txt

# a modified gaussian splatting (+ depth, alpha rendering)
!git clone --recursive https://github.com/ashawkey/diff-gaussian-rasterization
%pip install ./diff-gaussian-rasterization

# simple-knn
%pip install ./simple-knn

# nvdiffrast
%pip install git+https://github.com/NVlabs/nvdiffrast/

# kiuikit
%pip install git+https://github.com/ashawkey/kiuikit

%pip install  dearpygui

In [ ]:
%cd /notebooks/dreamgaussian

In [ ]:
%pip install nest-asyncio gradio==3.47.0 

In [ ]:
import nest_asyncio
nest_asyncio.apply()

%run gradio_app2.py

In [ ]:
file_name = "cat"
file_extension="png"
name="ouput"

In [ ]:
# Jupyter Notebook Cell 2

try:
    file_name
except NameError:
    print("Please upload a file first using the widget above.")
    raise

### preprocess
# background removal and recentering, save rgba at 256x256
%run process.py tmp_data/{file_name}.{file_extension}

# save at a larger resolution
#%run process.py data/{file_name} --size 512

# process all jpg images under a dir
#%run process.py data

### training gaussian stage
# train 500 iters (~1min) and export ckpt & coarse_mesh to logs


# gui mode (supports visualizing training)
# %run main.py --config configs/image.yaml input=data/{file_name}_rgba.png save_path=name gui=True

# load and visualize a saved ckpt
# %run main.py --config configs/image.yaml load=logs/name_model.ply gui=True

# use an estimated elevation angle if image is not front-view (e.g., common looking-down image can use -30)
# %run main.py --config configs/image.yaml input=data/{file_name}_rgba.png save_path=name elevation=-30

### training mesh stage
# auto load coarse_mesh and refine 50 iters (~1min), export fine_mesh to logs
#%run main2.py --config configs/image.yaml input=data/{file_name}_rgba.png save_path=name

# specify coarse mesh path explicity
#%run main2.py --config configs/image.yaml input=data/{file_name}_rgba.png save_path=name mesh=logs/name_mesh.obj

# gui mode
#%run main2.py --config configs/image.yaml input=data/{file_name}_rgba.png save_path=name gui=True

# export glb instead of obj



In [ ]:

%autoawait True
import nest_asyncio
nest_asyncio.apply()
import subprocess
import sys

python_exec_path = sys.executable
# Define the command and arguments
command = [
    f'{python_exec_path}', 'main.py',
    '--config', 'configs/image.yaml',
    f'input=tmp_data/{file_name}_rgba.png',
    'save_path=name'
]

# Start the subprocess with stdout and stderr redirected to pipes
process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

# Define a function to output the subprocess's log data in real-time
def stream_log(pipe):
    while True:
        # Read a line of output from the pipe
        line = pipe.readline()
        if line:
            # Print the line to the Jupyter Notebook's output
            print(line, end='')
        else:
            # No more output, break the loop
            break

# Call the function to stream the log data from the subprocess's stdout
stream_log(process.stdout)

# Wait for the subprocess to finish, if it hasn't already
return_code = process.wait()

# Check the return code to determine whether the subprocess succeeded or failed
if return_code == 0:
    print('Success')
else:
    print(f'Error: subprocess failed with return code {return_code}')

In [ ]:
%autoawait True
import nest_asyncio
nest_asyncio.apply()
import subprocess
import sys

python_exec_path = sys.executable
# Define the command and arguments
command = [
    f'{python_exec_path}', 'main2.py',
    '--config', 'configs/image_re.yaml',
    f'input=tmp_data/{file_name}_rgba.png',
    'save_path=name'
]

# Start the subprocess with stdout and stderr redirected to pipes
process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

# Define a function to output the subprocess's log data in real-time
def stream_log(pipe):
    while True:
        # Read a line of output from the pipe
        line = pipe.readline()
        if line:
            # Print the line to the Jupyter Notebook's output
            print(line, end='')
        else:
            # No more output, break the loop
            break

# Call the function to stream the log data from the subprocess's stdout
stream_log(process.stdout)

# Wait for the subprocess to finish, if it hasn't already
return_code = process.wait()

# Check the return code to determine whether the subprocess succeeded or failed
if return_code == 0:
    print('Success')
else:
    print(f'Error: subprocess failed with return code {return_code}')